In [1]:
from pyspark.sql import SparkSession
import getpass 
username = getpass.getuser()
spark = SparkSession. \
     builder. \
     config('spark.ui.port', '0'). \
     config("spark.sql.warehouse.dir", f"/user/itv009198/warehouse"). \
     config('spark.shuffle.useOldFetchProtocol', 'true'). \
     enableHiveSupport(). \
     master('yarn'). \
     getOrCreate()

In [2]:
raw_df = spark.read \
.format("csv") \
.option("inferSchema","true") \
.option("header","true") \
.load("/public/trendytech/datasets/accepted_2007_to_2018Q4.csv")

In [3]:
raw_df.createOrReplaceTempView("lending_club_data")

In [4]:
spark.sql ("select * from lending_club_data")

id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
68407277,null,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingcl...,null,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,null,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723916800001,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,null,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,null,null,null,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,null,null,null,null,null,null,null,null,null,null,null,null,null,N,null,null,null,null,null,null,null,null,null,null,null,null,null,null,Cash,N,null,null,null,null,null,null
68355089,null,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingcl...,null,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,null,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.66,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,null,Mar-2019,699.0,695.0,0.0,null,1.0,Individual,null,null,null,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,null,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,null,null,null,null,null,null,null,null

In [5]:
from pyspark.sql.functions import sha2, concat_ws

In [6]:
new_df = raw_df.withColumn("name_sha2",sha2(concat_ws("||",*["emp_title","emp_length","home_ownership","annual_inc","zip_code","addr_state","grade","sub_grade","verification_status"]),256))

In [7]:
new_df.createOrReplaceTempView("newtable")

In [8]:
spark.sql("select count(*)from newtable")

count(1)
2260701


In [9]:
spark.sql("select count(distinct(name_sha2))from newtable")

count(DISTINCT name_sha2)
2257384


In [10]:
spark.sql("""select name_sha2, count(*) as total_cnt
from newtable group by name_sha2 having total_cnt > 1 order by total_cnt desc""")

name_sha2,total_cnt
e3b0c44298fc1c149...,33
e4c167053d5418230...,5
3f87585a20f702838...,4
76b577467eda5bdbc...,4
ad8e5d384dae17e06...,4
059d401bb603d9a80...,3
53789bea7edc660ed...,3
5d52e7773cb0efff3...,3
3ae415acd6bbfaac1...,3
e7d8d16928817ec8f...,3


In [11]:
spark.sql("select * from newtable where name_sha2 like'e3b0c44298fc1c149%'")

id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,name_sha2
Total amount fund...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,e3b0c44298fc1c149...
Total amount fund...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,e3b0c

In [12]:
spark.sql("""select name_sha2 as member_id, emp_title, emp_length, home_ownership,
annual_inc, addr_state, zip_code, 'USA' as country, grade, sub_grade,
verification_status, tot_hi_cred_lim, application_type, annual_inc_joint, verification_status_joint
from newtable""").repartition(1).write \
.option("header",True) \
.format("csv") \
.mode("overwrite") \
.option("path","public/trendytech/datasets/lendingclubproject/raw/customers_data_csv") \
.save()

In [13]:
customers_df = spark.read \
.format("csv") \
.option("inferSchema","true") \
.option("header","true") \
.load("public/trendytech/datasets/lendingclubproject/raw/customers_data_csv")

In [14]:
customers_df

member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint
5d7676571dee53d06...,CNC Machinist,8 years,MORTGAGE,50000.0,SD,571xx,USA,C,C2,Not Verified,226297.0,Individual,null,null
31fd1ef036c5caf26...,null,null,OWN,34200.0,OH,458xx,USA,B,B1,Verified,93184.0,Individual,null,null
9b48253ca5848fa78...,partner,10+ years,MORTGAGE,200000.0,NY,120xx,USA,B,B3,Verified,80595.0,Individual,null,null
d6b8f2e32be148721...,dup. equip. opr,10+ years,RENT,33000.0,CA,908xx,USA,F,F1,Source Verified,18681.0,Individual,null,null
0a81ad556e20bcdb9...,PRODUCTION MANAGER,10+ years,MORTGAGE,63000.0,KS,670xx,USA,B,B5,Verified,178551.0,Individual,null,null
843b995d39160ddab...,POLISHER,10+ years,RENT,32000.0,OH,445xx,USA,B,B5,Source Verified,67635.0,Individual,null,null
04a8bd7c4e1407ba8...,distribution,6 years,MORTGAGE,41000.0,MI,481xx,USA,D,D2,Source Verified,53500.0,Individual,null,null
3e9a89fa96988af68...,Service Tech,10+ years,MORTGAGE,43520.0,TN,372xx,USA,D,D3,Verified,130051.0,Individual,null,null
394a8db0bd3bcf7de...,registered nurse,10+ years,MORTGAGE,83000.0,VA,232xx,USA,B,B1,Not Verified,132267.0,Individual,null,null
09cea8d2b6f1272f7...,null,null,RENT,22000.0,NY,103xx,USA,E,E2,Source Verified,9600.0,Individual,null,null


In [15]:
spark.sql("""select id as load_id, name_sha2 as member_id,loan_amnt,funded_amnt
term,int_rate,installment,issue_d,loan_status,purpose,title
from newtable""").repartition(1).write \
.option("header",True) \
.format("csv") \
.mode("overwrite") \
.option("path","public/trendytech/datasets/lendingclubproject/raw/loans_data_csv") \
.save()

In [16]:
loans_df = spark.read \
.format("csv") \
.option("inferSchema","true") \
.option("header","true") \
.load("public/trendytech/datasets/lendingclubproject/raw/loans_data_csv")

In [17]:
loans_df

load_id,member_id,loan_amnt,term,int_rate,installment,issue_d,loan_status,purpose,title
40391322,4480925324607267c...,35000.0,35000.0,19.99,927.1,Feb-2015,Fully Paid,debt_consolidation,Debt consolidation
40461475,b54711d4a553ea330...,5600.0,5600.0,14.31,192.24,Feb-2015,Charged Off,debt_consolidation,Debt consolidation
40361493,db06b45a938f1a3b5...,2925.0,2925.0,16.49,103.55,Feb-2015,Fully Paid,credit_card,Credit card refin...
38615026,ad9d9524477e85c11...,12300.0,12300.0,7.49,382.55,Feb-2015,Fully Paid,credit_card,Credit card refin...
40451319,c67f6ac3fea6ef46d...,12000.0,12000.0,10.49,389.98,Feb-2015,Fully Paid,debt_consolidation,Debt consolidation
40287423,bb36e2cb69517fac3...,11625.0,11625.0,19.99,431.97,Feb-2015,Charged Off,debt_consolidation,Debt consolidation
39987662,af69a7dff814fb213...,6500.0,6500.0,14.99,225.3,Feb-2015,Fully Paid,debt_consolidation,Debt consolidation
40360143,c9c794b5025e14a7d...,20950.0,20950.0,17.86,755.93,Feb-2015,Charged Off,debt_consolidation,Debt consolidation
40510540,61b48d763bd82369a...,22000.0,22000.0,9.49,704.63,Feb-2015,Fully Paid,credit_card,Credit card refin...
40581708,adc390ceaa6428ba4...,5000.0,5000.0,12.99,168.45,Feb-2015,Fully Paid,debt_consolidation,Debt consolidation


In [18]:
spark.sql("""select id as loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,
total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d
from newtable""").repartition(1).write \
.option("header",True) \
.format("csv") \
.mode("overwrite") \
.option("path","public/trendytech/datasets/lendingclubproject/raw/loans_repayments_data_csv") \
.save()

In [19]:
loans_repayments_df = spark.read \
.format("csv") \
.option("inferSchema","true") \
.option("header","true") \
.load("public/trendytech/datasets/lendingclubproject/raw/loans_repayments_data_csv")

In [20]:
loans_repayments_df

loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d
76003861,6423.8,5607.85,0.0,12031.65,574.88,Jan-2018,null
76263914,2400.0,323.18,0.0,2723.1800875923,905.18,Dec-2017,null
75537401,12600.0,1392.76,0.0,13992.7563947386,3437.74,Aug-2018,null
75038986,7805.03,7583.71,0.0,15388.74,440.72,Mar-2019,Apr-2019
76301424,2614.78,1080.09,0.0,3694.87,153.89,Apr-2018,null
75333198,8664.09,3431.32,68.8,12164.21,343.9,Mar-2019,Apr-2019
76391453,35000.0,10266.01,0.0,45266.0062708179,18884.35,Feb-2019,null
76363364,15000.0,2093.11,0.0,17093.108291696,4648.59,Aug-2018,null
76272510,14463.21,11178.79,110.13,25752.13,734.18,Mar-2019,Apr-2019
76304116,4800.0,1051.65,0.0,5851.6523334103,1761.59,Dec-2017,null


In [21]:
spark.sql("""select name_sha2 as member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,
total_rec_late_fee, mths_since_last_delinq, mths_since_last_record 
from newtable""").repartition(1).write \
.option("header",True) \
.format("csv") \
.mode("overwrite") \
.option("path","public/trendytech/datasets/lendingclubproject/raw/loans_defaulters_csv") \
.save()

In [22]:
loans_defaulters_df = spark.read \
.format("csv") \
.option("inferSchema","true") \
.option("header","true") \
.load("public/trendytech/datasets/lendingclubproject/raw/loans_defaulters_csv")

In [23]:
loans_defaulters_df

member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record
b59d80da191f5b573...,0.0,0.0,0.0,0.0,1.0,0.0,31.0,null
202d9f56ecb7c3bc9...,1.0,0.0,0.0,0.0,0.0,0.0,6.0,null
e5a140c0922b554b9...,0.0,0.0,0.0,0.0,0.0,0.0,47.0,null
e12aefc548f750777...,0.0,0.0,0.0,0.0,0.0,0.0,33.0,null
1b3a50d854fbbf97e...,1.0,0.0,0.0,0.0,0.0,0.0,21.0,null
1c4329e5f17697127...,0.0,0.0,0.0,0.0,0.0,0.0,null,null
5026c86ad983175eb...,0.0,0.0,1.0,0.0,2.0,0.0,null,71.0
9847d8c1e9d0b2084...,1.0,0.0,2.0,0.0,0.0,0.0,6.0,63.0
8340dbe1adea41fb4...,0.0,0.0,0.0,0.0,0.0,0.0,36.0,null
d4de0de3ab7d79ad4...,0.0,0.0,0.0,0.0,0.0,0.0,35.0,null
